In [1]:
!pip install split-folders

In [2]:
!pip install efficientnet_pytorch

  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet_pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16428 sha256=d53c18e40bc8c4495380ed64dd352f420a512fb00b2cd2f8082b7616a116d062
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
Successfully built efficientnet_pytorch


In [3]:
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision.transforms import transforms
import os
from PIL import Image
import torch
import ssl
import torchvision
import torchvision.models as models
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import splitfolders
import csv
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from efficientnet_pytorch import EfficientNet
import warnings

warnings.filterwarnings(
    action='ignore')


def calculate_accuracy(y_pred, y):
    top_pred = y_pred.argmax(1, keepdim=True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

ssl._create_default_https_context = ssl._create_unverified_context

if __name__ == '__main__':
    # Define the device to be used for training
    device = torch.device("cpu")

    # Set up the transform to resize and normalize the images
    transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5],
            std=[0.5, 0.5, 0.5]),
    ])
     
   # loading the temp.zip and creating a zip object 
  # with ZipFile("C:\\Users\\sai mohan pulamolu\\\ 
   #         Desktop\\geeks_dir\\temp\\temp.zip", 'r') as zObject: 
  
    # Extracting all the members of the zip  
    # into a specific location. 
   # zObject.extractall( 
    #    path="C:\\Users\\sai mohan pulamolu\\Desktop\\geeks_dir\\temp") 
    # Update input folder and output folder paths
    input_folder = r"/kaggle/input/base-dataset-without-preprocessing/Without_any_preprocessing_small"
    output_folder = r"/kaggle/working/images_30_output"
    

    ### Uncomment only for first time. once data is splitted into train and validation, comment it out
    splitfolders.ratio(input_folder, output_folder, seed=42, ratio=(0.8, 0.2), group_prefix=None)

    # Create datasets for the training and testing sets
    train_dataset = torchvision.datasets.ImageFolder(output_folder + '/train', transform=transform)
    val_dataset = torchvision.datasets.ImageFolder(output_folder + '/val', transform=transform)
    train_size = len(train_dataset)
    val_size = len(val_dataset)

    # Create the data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=25, shuffle=True,num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=25, shuffle=True,num_workers=4)
    list_of_classes = os.listdir(r"/kaggle/working/images_30_output/train")
    print(list_of_classes)
    classes = list(train_dataset.class_to_idx.keys())
    classes.sort()


    # Define the VGG model
    model_name = 'efficientnet-b0'  # You can change this to other versions (b1, b2, etc.)
    model = EfficientNet.from_pretrained(model_name)
    num_features = model._fc.in_features
    model._fc = nn.Linear(num_features, len(list_of_classes))

  

    ## uncomment for CPU
    model = model.to(device)

    # Define the loss function and optimizer
    
    
    criterion = nn.CrossEntropyLoss()
    START_LR = 0.0001
    optimizer = optim.Adam(model.parameters(), lr=START_LR)
    model = model.to(device)
    criterion = criterion.to(device)

    ### uncomment for GPU
    #model.cuda()
        # Train the model
    for epoch in range(10):
        ### uncomment for GPU
        #torch.cuda.empty_cache()
        print('Epoch {}/{}'.format(epoch + 1, 10))
        print('-' * 10)

        running_loss = 0
        running_corrects = 0

        model.train()
        predictions = []
        true_labels = []
        for inputs, labels in tqdm(train_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())
            optimizer.zero_grad()
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / train_size
        epoch_acc = running_corrects.double() / train_size
        report_dict = classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_pd = pd.DataFrame(report_dict)
        report_pd.to_csv('training-classification-epoch' + str(epoch + 1) + '.csv')
        cnf_matrix = confusion_matrix(true_labels, predictions)
        df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index = [i for i in classes],
                        columns = [i for i in classes])
        df_cm.to_csv('confusion-matrix-train-epoch' + str(epoch + 1) + '.csv')
        #acc = matrix.diagonal()/matrix.sum(axis=1)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        ACC = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        PPV = TP/(TP+FP)
        print("accuracy for all classes in train phase", ACC)
        print("recall for all classes in train phase", TPR)
        print("precision for all classes in train phase", PPV)
        
        pd.DataFrame(ACC, columns=['Accuracy']).to_csv('accuracy-train-epoch' + str(epoch + 1) + '.csv')
        print('Train Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        # Delete data to clear GPU memory
        del outputs
        del preds
        del labels
        del inputs
        torch.cuda.empty_cache()
        
        # Validation phase
        running_loss = 0
        running_corrects = 0
        model.eval()  # set the model to evaluation mode
        predictions = []
        true_labels = []
        for inputs, labels in tqdm(val_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            true_labels.extend(labels.cpu().numpy())
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            predictions.extend(preds.cpu().numpy())
            loss = criterion(outputs, labels)
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
        epoch_loss = running_loss / val_size
        epoch_acc = running_corrects.double() / val_size
       
        #classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_dict = classification_report(true_labels, predictions, target_names=list_of_classes,output_dict=True)
        report_pd = pd.DataFrame(report_dict)
        report_pd.to_csv('val-classification-epoch' + str(epoch + 1) + '.csv')
        #matrix = confusion_matrix(true_labels, predictions)
        cnf_matrix = confusion_matrix(true_labels, predictions)
        df_cm = pd.DataFrame(cnf_matrix / np.sum(cnf_matrix, axis=1)[:, None], index = [i for i in classes],
                        columns = [i for i in classes])
        df_cm.to_csv('confusion-matrix-val-epoch' + str(epoch + 1) + '.csv')
        #acc = matrix.diagonal()/matrix.sum(axis=1)
        FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix) 
        FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
        TP = np.diag(cnf_matrix)
        TN = cnf_matrix.sum() - (FP + FN + TP)
        FP = FP.astype(float)
        FN = FN.astype(float)
        TP = TP.astype(float)
        TN = TN.astype(float)
        ACC = (TP+TN)/(TP+FP+FN+TN)
        TPR = TP/(TP+FN)
        PPV = TP/(TP+FP)
        pd.DataFrame(ACC, columns=['Accuracy']).to_csv('accuracy-val-epoch' + str(epoch + 1) + '.csv')
        # print("accuracy for all classes in validation phase", ACC)
        # print("recall for all classes in validation phase", TPR)
        # print("precision for all classes in validation phase", PPV)
        print('Val Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
        # Delete data to clear GPU memory
        del outputs
        del preds
        del labels
        del inputs
        torch.cuda.empty_cache()
                
        

    # Save the model
    torch.save(model, 'efficientnet_model.pth')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
Copying files: 2225 files [00:16, 137.57 files/s]
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth


['blakit1', 'abhori1', 'chespa1', 'whctur2', 'whbcan1', 'combul2', 'somgre1', 'spwlap1', 'gobwea1', 'yebsto1', 'chtapa3', 'vilwea1', 'reccuc1', 'whbtit5', 'nubwoo1', 'slbgre1', 'spewea1', 'colsun2', 'gobbun1', 'norpuf1', 'brican1', 'pygbat1', 'grbcam1', 'palfly2', 'carcha1', 'rebfir2', 'afrjac1', 'chibat1', 'afghor1', 'grewoo2']


100%|██████████| 20.4M/20.4M [00:00<00:00, 206MB/s]


Loaded pretrained weights for efficientnet-b0
Epoch 1/10
----------


100%|██████████| 71/71 [05:09<00:00,  4.36s/it]


...

Val Loss: 1.0694 Acc: 0.7177
Epoch 10/10
----------


100%|██████████| 71/71 [04:10<00:00,  3.53s/it]


accuracy for all classes in train phase [0.99208145 0.99660633 0.9949095  0.99321267 0.99434389 0.99208145
 0.99886878 0.99151584 0.99773756 0.99264706 0.98642534 0.99660633
 0.99773756 0.99434389 0.99547511 0.99434389 0.99660633 0.99434389
 0.99717195 0.99264706 0.99773756 0.99660633 0.99434389 0.99660633
 0.99547511 0.99377828 0.99943439 0.99547511 0.99886878 0.99943439]
recall for all classes in train phase [0.95       0.92982456 0.83333333 0.98564593 0.86956522 0.95901639
 0.5        0.88888889 0.9047619  0.99305556 0.96581197 0.9375
 0.         0.94666667 0.98780488 0.5        0.76470588 0.09090909
 0.81818182 0.81081081 0.97938144 0.66666667 0.98113208 0.76
 0.89361702 0.92857143 0.875      0.55555556 0.75       0.75      ]
precision for all classes in train phase [0.91346154 0.96363636 0.8        0.95813953 0.74074074 0.92857143
 1.         0.875      0.9047619  0.92258065 0.9338843  0.96774194
        nan 0.92207792 0.92045455 0.8        0.86666667 1.
 0.75       0.83333333 0.9

100%|██████████| 19/19 [00:25<00:00,  1.36s/it]

Val Loss: 1.0194 Acc: 0.7309
